###  mmult on PYNQ
##### Source : https://github.com/tkat0/pynqmmult

* SDSoC付属の内積演算サンプルをPYNQで動かしてみた
* ドライバは共有ライブラリとしてビルドし、CFFIを利用してPythonから呼び出す
* CMAでアロケートした連続領域をnumpyのndarrayとして扱えるようにした
* 開発環境は、SDSoC 2015.4

In [1]:
import numpy as np
import cffi
from pynq import Overlay
# load Base Overlay
Overlay("/home/xilinx/pynq/bitstream/base.bit").download()

from pynq.drivers import xlnk

In [2]:
# SDSoCでビルドした共有ライブラリから、HW化した関数の名前を把握する
# SW版の関数名は、mmult_accelだが、HW化すると_p0_mmult_accel_0となる。
# さらに、コンパイラの名前マングリングにより_Z17_p0_mmult_accel_0PfS_S_になるよう
!nm -C ./libpynqmmult.so | grep mmult_accel
!echo ---
!nm -D ./libpynqmmult.so | grep mmult_accel

nm: './libpynqmmult.so': No such file
---
nm: './libpynqmmult.so': No such file


### Call Accelerator

In [3]:
class Mmult():
    def __init__(self):
        self.bitfile = "./pynqmmult.bit"
        self.libfile = "./libpynqmmult.so"
        self.ffi = cffi.FFI()
        self.ffi.cdef("void _Z17_p0_mmult_accel_0PfS_S_(float*, float*, float*);")
        self.lib = self.ffi.dlopen(self.libfile)
        Overlay(self.bitfile).download()
    
    def __call__(self, a, b, c):
        # a,b,c is CData Object
        self.lib._Z17_p0_mmult_accel_0PfS_S_(a,b,c)


In [4]:
mmult = Mmult()

OSError: cannot load library ./libpynqmmult.so: ./libpynqmmult.so: cannot open shared object file: No such file or directory

In [ ]:
# SDSoCにより高位合成したHWにDMA(not SG)経由で読み書きするメモリ領域は連続領域である必要がある
# 連続領域は、CMAのAPIにより、確保する
# 連続領域をndarrayとして扱う

memmanager = xlnk.xlnk()
ffi = cffi.FFI()

 # TODO 現状32x32の配列のみ対応。汎用化する
def init_contiguous_ndarray(size=(32,32), dtype="float"):
    buf = memmanager.cma_alloc(32*32, data_type=dtype)
    cbuf = ffi.buffer(buf,  32*32 * ffi.sizeof(dtype))
    return np.frombuffer(cbuf, dtype=np.float32).reshape(size), buf

In [ ]:
a, pa = init_contiguous_ndarray()
b, pb = init_contiguous_ndarray()
c, pc = init_contiguous_ndarray()

a += 1
b += 2

print("A", a.shape, type(a))
print(a)
print("B", b.shape, type(b))
print(b)
print("C", c.shape, type(c))
print(c)

In [ ]:
mmult(pa, pb, pc)

In [ ]:
print("C", c.shape, type(c))
print(c)

### Test

In [ ]:
# For comparison
np.dot(a,b)

In [ ]:
# SWとHWの計算結果の全ての要素が一致することを確認
if np.alltrue(c == np.dot(a, b)):
    print("OK")
else:
    print("NG")

### Benchmarks

In [ ]:
# HW
t_hw = %timeit -n 100 -o mmult(pa, pb, pc)

In [ ]:
# SW
t_sw = %timeit -n 100 -o np.dot(a, b)